In [8]:
import pandas as pd
import numpy as np
import gzip
import json
import os
import yaml
from tqdm import tqdm
import re

In [9]:
# Code to take a peak at the data structure

file_path = './anonymized/data/1669400411-a21252c1-761a-4594-b463-9892bedd8298.jsonl.gz'

logs = []
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            event = json.loads(line)
            if event.get("event_type") == "combat_state_update":
                combat = event.get("data")
                for actor in combat.get("combatants", []):
                    if actor["type"] == "monster":
                        logs.append(actor)
                        
logs[1]

{'name': 'Cannon Protector',
 'stats': {'prof_bonus': 0,
  'strength': 10,
  'dexterity': 10,
  'constitution': 10,
  'intelligence': 10,
  'wisdom': 10,
  'charisma': 10},
 'levels': {'total_level': 0.125, 'classes': {'Monster': 0.125}},
 'attacks': [],
 'skills': {'acrobatics': {'value': 0},
  'animalHandling': {'value': 0},
  'arcana': {'value': 0},
  'athletics': {'value': 0},
  'deception': {'value': 0},
  'history': {'value': 0},
  'initiative': {'value': 0},
  'insight': {'value': 0},
  'intimidation': {'value': 0},
  'investigation': {'value': 0},
  'medicine': {'value': 0},
  'nature': {'value': 0},
  'perception': {'value': 0},
  'performance': {'value': 0},
  'persuasion': {'value': 0},
  'religion': {'value': 0},
  'sleightOfHand': {'value': 0},
  'stealth': {'value': 0},
  'survival': {'value': 0},
  'strength': {'value': 0},
  'dexterity': {'value': 0},
  'constitution': {'value': 0},
  'intelligence': {'value': 0},
  'wisdom': {'value': 0},
  'charisma': {'value': 0}},
 

In [ ]:
# Main function that parses each combat session

from collections import defaultdict
combat_data = defaultdict(lambda: {
     'start_time': None, 
     'player_ids': set(),
     'player_info': {} 
})
def process_file(file_path):
    monsters_found = False
    players_found = False
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            event = json.loads(line)
            if event.get("event_type") == "combat_state_update":
                combat = event.get("data")
                for actor in combat.get("combatants", []):
                    if actor["type"] == "monster":
                        monsters_found = True
                    elif actor["type"] == "player":
                        players_found = True
                    if monsters_found and players_found:
                        break  # Break from the inner loop if both found
                if monsters_found and players_found:
                    break  # Break from the outer loop if both found

    # If no monsters or players were found, return immediately
    if not monsters_found or not players_found:
        return None  # Or an appropriate indicator that the file was skipped
    
    last_human_readable = {}

    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            event = json.loads(line)
            combat_id = event.get("combat_id")

            # Checking for combat start and getting timestamp (dont think we actually need timestamp)
            if event.get("event_type") == "combat_start":
                combat_data[combat_id]['start_time'] = event.get("timestamp")

            # counting players who joined initiative and counting spell slots
            #elif event.get("event_type") == "command" and event.get("command_name") == "init join":
              #  player_id = event.get("author_id")
              #  player_name = event.get("caster", {}).get("name", event.get("author_name"))
              #  combat_data[combat_id]['player_ids'].add(player_id)
              #  combat_data[combat_id]['player_info'][player_name] = {'hp_ratio': None}
              #  classes_dict = event['caster']['levels']['classes']
              #  combat_data[combat_id]['player_info'][player_name]['class'] = list(classes_dict.items())
            elif event.get("event_type") == "command" and event.get("command_name") == "init join":
                player_id = event.get("author_id")
                player_name = event.get("caster", {}).get("name", event.get("author_name"))
                combat_data[combat_id]['player_ids'].add(player_id)
                classes_dict = event['caster']['levels']['classes']
                combat_data[combat_id]['player_info'][player_name] = {
                    'hp_ratio': None,
                    'class': list(classes_dict.items()),
                    'slots': event['caster']['spellbook']['slots'],
                    'max_slots': event['caster']['spellbook']['max_slots'],
                    'ac': event['caster']['ac'],  # Adding AC value
                    'stats': event['caster']['stats']  # Adding stats
                }

                # Initialize slot sums 
                if 'total_slots' not in combat_data[combat_id]:
                    combat_data[combat_id]['total_slots'] = defaultdict(int)
                    combat_data[combat_id]['total_max_slots'] = defaultdict(int)

                # Summing slots and max_slots
                for slot, value in event['caster']['spellbook']['slots'].items():
                    combat_data[combat_id]['total_slots'][slot] += value

                for slot, value in event['caster']['spellbook']['max_slots'].items():
                    combat_data[combat_id]['total_max_slots'][slot] += value

            elif event.get("event_type") == "combat_state_update":
                combat = event.get("data")
                for actor in combat.get("combatants", []):
                    if actor["type"] == "monster":
                        # Ensure 'monsters' is initialized as a list if it doesn't already exist
                        if 'monsters' not in combat_data[combat_id]:
                            combat_data[combat_id]['monsters'] = []

                        # Define a helper function to check if the monster is already added
                        def monster_exists(monster_list, monster_id):
                            return any(monster['monster_id'] == monster_id for monster in monster_list)

                        # Check if this monster is already added, to avoid duplication
                        if not monster_exists(combat_data[combat_id]['monsters'], actor["id"]):
                            # If the monster is not in the list, add it with all its details
                            combat_data[combat_id]['monsters'].append({
                                'monster_id': actor["id"],
                                'monster_code': actor["name"],
                                'monster_name': actor['monster_name'],
                                'level': actor['levels']['total_level']
                            })


                #trying to get that human readable part
                human_readable = event.get("human_readable")

                if human_readable:
                    last_human_readable[combat_id] = human_readable
  
    # Grab player names from combat_data dictionary
    player_names = ', '.join(str(key) for key in combat_data[combat_id]['player_info'].keys())
    if len(player_names) != 0:
        player_names = player_names.split(',')
        player_names = [name.lstrip() for name in player_names]

    


    if last_human_readable[combat_id]:
        # print(last_human_readable[combat_id])

        # Iterating through player names and finding their health in the human_readable string
        if len(player_names) != 0:
            for player in player_names:
                try: 
                    pattern = rf"{player} <(\d+)/(\d+) HP>"

                    match = re.search(pattern, last_human_readable[combat_id])

                    if match:
                        current_hp, max_hp = match.groups()

                        if player in combat_data[combat_id]['player_info'] and current_hp and max_hp:
                            combat_data[combat_id]['player_info'][player]['hp_ratio'] = [int(current_hp), int(max_hp)]
                        else:
                            continue
                except re.error:
                    print(f"Skipping due to an error with pattern: {pattern}")
                    continue


#     # Processing the last human_readable to extract HP ratios (this is not working ....)
#     for cid, human_readable in last_human_readable.items():
#         pattern = r'\d+: ([\w\s.-]+) <(\d+/\d+ HP)>'
#         matches = re.findall(pattern, human_readable)

#         for name, hp_ratio in matches:
#             if name in combat_data[cid]['player_info']:
#                 combat_data[cid]['player_info'][name]['hp_ratio'] = hp_ratio

#     # Printing the last human_readable string for each combat ID
#     for cid, human_readable in last_human_readable.items():
#         print(f"Last 'human_readable' for {cid}: {human_readable}")

    # # Creating the DataFrame from collected data
    # processed_data = []
    # for cid, data in combat_data.items():
    #     processed_data.append({
    #         "combat_id": cid,
    #         "start_time": data['start_time'],
    #         "num_player_actors": len(data['player_ids']),
    #         "player_info": data['player_info'],
    #         "num_monster_actors": len(data['monster_ids']),
    #         "monster_ids": list(data['monster_ids']),
    #         "monster_names": list(data['monster_names']),
    #     })

    # return pd.DataFrame(processed_data)
    return combat_data
    
# Define a file path
file_path = './anonymized/data/1669400411-a21252c1-761a-4594-b463-9892bedd8298.jsonl.gz'

# Process the file and create DataFrame
combat_data = process_file(file_path)

# # Export to CSV
# csv_file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/combat_analysis_with_hp.csv'
# df.to_csv(csv_file_path, index=False)

# print(f"Data exported to CSV file at: {csv_file_path}")

combat_data

In [ ]:
# Function that takes the above function and iterates it over a directory of files

directory_path = './anonymized/data/'
files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.jsonl.gz')]
#files = files[:50]
def process_combat_files(file_paths):
    combat_datas = {}  # This will store the combined data from all files

    for file_path in tqdm(file_paths, desc="Processing files"):
        # Process each file to get its combat data
        combat_data_latest = process_file(file_path)
        
        # If combat_data_latest is None (file was skipped), continue to the next file
        if combat_data_latest is None:
            continue

        # Merge the data into combat_datas
        # Assuming combat_data_latest contains data keyed by combat_id
        for combat_id, data in combat_data_latest.items():
            if combat_id in combat_datas:
                # If combat_id already exists, merge or update data as needed
                # This part depends on how you want to handle duplicate combat_ids
                # For simplicity, let's just update the existing data with the new one
                combat_datas[combat_id].update(data)
            else:
                # Add the new combat_id and its data to combat_datas
                combat_datas[combat_id] = data

    return combat_datas


final_combat_data = process_combat_files(files)


In [ ]:
final_combat_data[list(final_combat_data.keys())[np.random.randint(0,len(final_combat_data))]] # Randomly selecting an item from the list to see if the function is working properly

In [ ]:
len(final_combat_data)

In [ ]:
from tqdm import tqdm
import numpy as np

def calculate_averages_for_encounters(encounters_dict):
    monster_levels = []  # List to accumulate monster levels
    player_healths = []  # List to accumulate player health percentages

    for encounter_name in tqdm(encounters_dict.keys(), desc="Calculating averages"):
        encounter_data = encounters_dict[encounter_name]

        # Accumulate monster levels
        total_level = sum(monster['level'] for monster in encounter_data['monsters'])
        monster_levels.append(total_level) if encounter_data['monsters'] else 0

        # Handle missing or None hp_ratio
        player_infos = encounter_data['player_info'].values()
        total_health_ratio = sum(player['hp_ratio'][0] for player in player_infos if player.get('hp_ratio'))
        total_max_health = sum(player['hp_ratio'][1] for player in player_infos if player.get('hp_ratio'))
        
        # Calculate and accumulate average health left over, handling division by zero
        if total_max_health > 0:
            player_healths.append(total_health_ratio / total_max_health * 100)
        else:
            player_healths.append(0)  # Default value or handle as appropriate

    # Convert lists to NumPy arrays for final calculation
    monster_levels_array = np.array(monster_levels)
    player_healths_array = np.array(player_healths)

    # Calculate overall averages
    monster_average = np.mean(monster_levels_array)
    player_average = np.mean(player_healths_array)

    return monster_levels, player_healths, monster_average, player_average


mls,phs, monster_average, player_average = calculate_averages_for_encounters(final_combat_data)

In [ ]:

final_combat_data[list(final_combat_data.keys())[0]]

In [ ]:
monster_average, player_average

## Dataframe Organization & .CSV Output

In [ ]:
# create dataframe from final dictionary
final_combat_data_df = pd.DataFrame.from_dict(final_combat_data)

# transpose dataframe to improve readability
final_combat_data_df_T = final_combat_data_df.transpose()

# name index column to combat_id
final_combat_data_df_T.index.name = 'combat_id'

final_combat_data_df_T.head()

### Column Parsing
#### Party/Player Information

In [ ]:
# add column for party size (read from number of player_ids)
player_number_list = []

for index, row in final_combat_data_df_T.iterrows():
  num_players_val = len(row['player_ids'])
  player_number_list.append(num_players_val)

final_combat_data_df_T['party_size']  = player_number_list

In [ ]:
# parsing player info (read from player_info)
# column for class and level
party_class_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_dict = row['player_info']
  party_classes_and_levels = [val.get('class') for val in row_dict.values()]
  party_class_list.append(party_classes_and_levels)

final_combat_data_df_T['party_classes_with_level']  = party_class_list

In [ ]:
# column for total party level (read from player_info)
party_total_level_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_dict = row['party_classes_with_level']
  party_total_level = 0
  for x in row_dict:
    for y in x:
      class_level = y[-1]
      party_total_level = party_total_level + class_level
  party_total_level_list.append(party_total_level)

final_combat_data_df_T['party_total_level']  = party_total_level_list

In [ ]:
# column for party class composition (read from player_info)
party_total_class_composition_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_dict = row['party_classes_with_level']
  party_class_composition_list = []
  for x in row_dict:
    for y in x:
      class_type = y[0]
      party_class_composition_list.append(class_type)
  party_total_class_composition_list.append(party_class_composition_list)

final_combat_data_df_T['party_total_class_composition']  = party_total_class_composition_list

In [ ]:
# column for HP ratios (by party member) (read from player_info)
player_individual_hpratio_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_dict = row['player_info']
  player_hpratio = [val.get('hp_ratio') for val in row_dict.values()]
  player_individual_hpratio_list.append(player_hpratio)

final_combat_data_df_T['player_individual_hp_ratios']  = player_individual_hpratio_list

In [ ]:
# column for party pre-combat HP (read from player_info)
party_total_precombat_hp_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_list = row['player_individual_hp_ratios']
  hp_precombat_sum = 0
  for hp_ratio in row_list:
    if hp_ratio == None:
      continue
    hp_precombat_val = int(hp_ratio[-1])
    hp_precombat_sum = hp_precombat_sum + hp_precombat_val
  party_total_precombat_hp_list.append(hp_precombat_sum)

final_combat_data_df_T['party_total_precombat_hp']  = party_total_precombat_hp_list
  

In [ ]:
# column for party post-combat HP (read from player_info)
party_total_postcombat_hp_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_list = row['player_individual_hp_ratios']
  hp_postcombat_sum = 0
  for hp_ratio in row_list:
    if hp_ratio == None:
      continue
    hp_postcombat_val = int(hp_ratio[0])
    hp_postcombat_sum = hp_postcombat_sum + hp_postcombat_val
  party_total_postcombat_hp_list.append(hp_postcombat_sum)

final_combat_data_df_T['party_total_postcombat_hp']  = party_total_postcombat_hp_list

In [ ]:
# column for party HP ratio (expressed as a fraction of post:pre combat HP)
party_total_hpratio_list = []

for index, row in final_combat_data_df_T.iterrows():
  hp_precombat_instance = row['party_total_precombat_hp']
  hp_postcombat_instance = row['party_total_postcombat_hp']
  if hp_postcombat_instance == 0:
    party_hp_ratio = 'NA'
  else: 
    party_hp_ratio = float(hp_postcombat_instance / hp_precombat_instance)
  party_total_hpratio_list.append(party_hp_ratio)

final_combat_data_df_T['party_total_hpratio']  = party_total_hpratio_list

In [ ]:
# column for armor class (ac) ratings for each individual in a party
player_individual_AC_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_dict = row['player_info']
    party_ac_values = [val.get('ac') for val in row_dict.values()]
    player_individual_AC_list.append(party_ac_values)   

final_combat_data_df_T['player_individual_ac']  = player_individual_AC_list


In [ ]:
# column for total party ac
party_total_AC_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_ac']
    party_ac_sum = sum(row_list)
    party_total_AC_list.append(party_ac_sum)   

final_combat_data_df_T['party_total_ac']  = party_total_AC_list


In [ ]:
# column for party ability scores (by player), grouped by ability
player_individual_prof_bonus_scores_total_list = []
player_individual_strength_scores_total_list = []
player_individual_dex_scores_total_list = []
player_individual_con_scores_total_list = []
player_individual_int_scores_total_list = []
player_individual_wis_scores_total_list = []
player_individual_char_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_dict = row['player_info']
    player_stats = [val.get('stats') for val in row_dict.values()]
    player_individual_prof_bonus_scores = []
    player_individual_strength_scores = []
    player_individual_dex_scores = []
    player_individual_con_scores= []
    player_individual_int_scores = []
    player_individual_wis_scores= []
    player_individual_char_scores = []
    for dict in player_stats:
        player_prof_bonus_value = dict['prof_bonus']
        player_strength_value = dict['strength']
        player_dex_value = dict['dexterity']
        player_con_value = dict['constitution']
        player_int_value = dict['intelligence']
        player_wis_value = dict['wisdom']
        player_char_value = dict['charisma']
        player_individual_prof_bonus_scores.append(player_prof_bonus_value)
        player_individual_strength_scores.append(player_strength_value)
        player_individual_dex_scores.append(player_dex_value)
        player_individual_con_scores.append(player_con_value)
        player_individual_int_scores.append(player_int_value)
        player_individual_wis_scores.append(player_wis_value)
        player_individual_char_scores.append(player_char_value)

    player_individual_prof_bonus_scores_total_list.append(player_individual_prof_bonus_scores)
    player_individual_strength_scores_total_list.append(player_individual_strength_scores)
    player_individual_dex_scores_total_list.append(player_individual_dex_scores)
    player_individual_con_scores_total_list.append(player_individual_con_scores)
    player_individual_int_scores_total_list.append(player_individual_int_scores)
    player_individual_wis_scores_total_list.append(player_individual_wis_scores)
    player_individual_char_scores_total_list.append(player_individual_char_scores)
  
final_combat_data_df_T['player_individual_prof_bonus']  = player_individual_prof_bonus_scores_total_list
final_combat_data_df_T['player_individual_strength']  = player_individual_strength_scores_total_list
final_combat_data_df_T['player_individual_dexterity']  = player_individual_dex_scores_total_list
final_combat_data_df_T['player_individual_constitution']  = player_individual_con_scores_total_list
final_combat_data_df_T['player_individual_intelligence']  = player_individual_int_scores_total_list
final_combat_data_df_T['player_individual_wisdom']  = player_individual_wis_scores_total_list
final_combat_data_df_T['player_individual_charisma']  = player_individual_char_scores_total_list



In [ ]:
# column for total party prof bonus
party_prof_bonus_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_prof_bonus']
    party_prof_bonus_sum = sum(row_list)
    party_prof_bonus_scores_total_list.append(party_prof_bonus_sum)   

final_combat_data_df_T['party_total_prof_bonus']  = party_prof_bonus_scores_total_list


In [ ]:
# column for total party ability scores
## strength
party_strength_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_strength']
    party_strength_sum = sum(row_list)
    party_strength_scores_total_list.append(party_strength_sum)   

final_combat_data_df_T['party_total_strength']  = party_strength_scores_total_list

### dexterity
party_dex_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_dexterity']
    party_dex_sum = sum(row_list)
    party_dex_scores_total_list.append(party_dex_sum)   

final_combat_data_df_T['party_total_dexterity']  = party_dex_scores_total_list

### constitution
party_con_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_constitution']
    party_con_sum = sum(row_list)
    party_con_scores_total_list.append(party_con_sum)   

final_combat_data_df_T['party_total_constitution']  = party_con_scores_total_list

### intelligence
party_int_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_intelligence']
    party_int_sum = sum(row_list)
    party_int_scores_total_list.append(party_int_sum)   

final_combat_data_df_T['party_total_intelligence']  = party_int_scores_total_list

### wisdom
party_wis_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_wisdom']
    party_wis_sum = sum(row_list)
    party_wis_scores_total_list.append(party_wis_sum)   

final_combat_data_df_T['party_total_wisdom']  = party_wis_scores_total_list

### charisma
party_chr_scores_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_list = row['player_individual_charisma']
    party_chr_sum = sum(row_list)
    party_chr_scores_total_list.append(party_chr_sum)   

final_combat_data_df_T['party_total_charisma']  = party_chr_scores_total_list

In [ ]:
# column for all spell slots (seperated by slot level) (party total)
party_level1_spellslot_list = []
party_level2_spellslot_list = []
party_level3_spellslot_list = []
party_level4_spellslot_list = []
party_level5_spellslot_list = []
party_level6_spellslot_list = []
party_level7_spellslot_list = []
party_level8_spellslot_list = []
party_level9_spellslot_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_dict = row['total_max_slots']
    if type(row_dict) == float:
        party_level1_spellslot_list.append('NA')
        party_level2_spellslot_list.append('NA')
        party_level3_spellslot_list.append('NA')
        party_level4_spellslot_list.append('NA')
        party_level5_spellslot_list.append('NA')
        party_level6_spellslot_list.append('NA')
        party_level7_spellslot_list.append('NA')
        party_level8_spellslot_list.append('NA')
        party_level9_spellslot_list.append('NA')
    else:
        level1_sum = row_dict.get('1')
        level2_sum = row_dict.get('2')
        level3_sum = row_dict.get('3')
        level4_sum = row_dict.get('4')
        level5_sum = row_dict.get('5')
        level6_sum = row_dict.get('6')
        level7_sum = row_dict.get('7')
        level8_sum = row_dict.get('8')
        level9_sum = row_dict.get('9')
        party_level1_spellslot_list.append(level1_sum)
        party_level2_spellslot_list.append(level2_sum)
        party_level3_spellslot_list.append(level3_sum)
        party_level4_spellslot_list.append(level4_sum)
        party_level5_spellslot_list.append(level5_sum)
        party_level6_spellslot_list.append(level6_sum)
        party_level7_spellslot_list.append(level7_sum)
        party_level8_spellslot_list.append(level8_sum)
        party_level9_spellslot_list.append(level9_sum)


final_combat_data_df_T['party_level1_spellslots']  = party_level1_spellslot_list
final_combat_data_df_T['party_level2_spellslots']  = party_level2_spellslot_list
final_combat_data_df_T['party_level3_spellslots']  = party_level3_spellslot_list
final_combat_data_df_T['party_level4_spellslots']  = party_level4_spellslot_list
final_combat_data_df_T['party_level5_spellslots']  = party_level5_spellslot_list
final_combat_data_df_T['party_level6_spellslots']  = party_level6_spellslot_list
final_combat_data_df_T['party_level7_spellslots']  = party_level7_spellslot_list
final_combat_data_df_T['party_level8_spellslots']  = party_level8_spellslot_list
final_combat_data_df_T['party_level9_spellslots']  = party_level9_spellslot_list

#### Monster Information

In [ ]:
#rename monsters column to monsters_info
final_combat_data_df_T.rename(columns = {'monsters':'monsters_info'}, inplace = True) 

In [ ]:
# parsing monster info (read from monsters_info)
# column for monster types
monster_type_total_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_dict = row['monsters_info']
  monster_type_list = []
  for dic in row_dict:
    monster_type_list.append(dic['monster_name'])
  monster_type_total_list.append(monster_type_list)


final_combat_data_df_T['monster_types']  = monster_type_total_list

In [ ]:
# column for total number of monsters
monster_number_list = []

for index, row in final_combat_data_df_T.iterrows():
  row_dict = row['monster_types']
  num_monsters_val = len(row_dict)
  monster_number_list.append(num_monsters_val)

final_combat_data_df_T['monster_number']  = monster_number_list

In [ ]:
# column for monster total level
monster_level_total_list = []

for index, row in final_combat_data_df_T.iterrows():
    row_dict = row['monsters_info']
    monster_level_list = []
    monster_level_sum = 0
    for dic in row_dict:
        monster_level = float(dic['level'])
        monster_level_list.append(monster_level)
    monster_level_sum = sum(monster_level_list)
    monster_level_total_list.append(monster_level_sum)

final_combat_data_df_T['monster_total_level']  = monster_level_total_list

#### Check Dataframe & Export to CSV

In [ ]:
#check updated dataframe
final_combat_data_df_T.head()

In [ ]:
# # Export to CSV
csv_file_path = '/Users/noahwaller/Documents/final_combat_data_df_T_03-24_FULL.csv'
final_combat_data_df_T.to_csv(csv_file_path, index=True)